In [40]:
import pandas as pd
import numpy as np
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html
from dash import dcc
import plotly.express as px
from dash.dependencies import Input, Output
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.subplots as sp

In [41]:
# Insert data
df = pd.read_csv('olympic_data_country_medal_year.csv')
df.tail()

,Unnamed: 0,Country_Name,Country_Code,Year,Season,Population,GDP,GDP_Per_Capita,Y_Gold,Y_Silver,...,A_Bronze,A_Total,Y_Capita_Per_Medal,A_Capita_Per_Medal,Y_Capita_Per_GDP,A_Capita_Per_GDP,A_Score,Y_Score,Y_Score_Per_Capita,A_Score_Per_Capita
2150,2129,Ukraine,UKR,2020,Summer,44132049.0,1.655720e+11,3751.740723,1.0,6.0,...,72.0,147.0,4.305261e-07,3.330913e-06,1.147537e-10,8.878312e-10,260.0,27.0,1.634520e+06,169738.650000
2151,2075,United Kingdom,GBR,2020,Summer,67081000.0,2.704609e+12,40318.557570,22.0,20.0,...,193.0,523.0,9.540705e-07,7.796544e-06,2.366331e-11,1.933736e-10,1017.0,128.0,5.240703e+05,65959.685349
2152,2092,United States,USA,2020,Summer,331501080.0,2.106047e+13,63530.633480,39.0,41.0,...,523.0,1826.0,3.408737e-07,5.508278e-06,5.365501e-12,8.670270e-11,3836.0,232.0,1.428884e+06,86418.425443
2153,2130,Uzbekistan,UZB,2020,Summer,34232050.0,5.989431e+10,1749.655815,3.0,0.0,...,20.0,37.0,1.460620e-07,1.080858e-06,8.348039e-11,6.177549e-10,65.0,11.0,3.112005e+06,526646.923077
2154,2093,"Venezuela, RB",VEN,2020,Summer,28490453.0,0.000000e+00,0.000000,1.0,3.0,...,8.0,18.0,1.403979e-07,6.317906e-07,0.000000e+00,0.000000e+00,31.0,9.0,3.165606e+06,919046.870968


In [42]:
# Insert coordinates for participating Olympics
coordinates = pd.read_csv('Resources/country_coordinate.csv')
coordinates = coordinates.rename(columns={"name": "Country_Name"})
coordinates.head()

,country,latitude,longitude,Country_Name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [43]:
# Merge coordinates
df = pd.merge(df, coordinates, on ="Country_Name")
df = df.reset_index()
df= df.sort_values("Year", ascending = True)
df.tail()

,index,Unnamed: 0,Country_Name,Country_Code,Year,Season,Population,GDP,GDP_Per_Capita,Y_Gold,...,A_Capita_Per_Medal,Y_Capita_Per_GDP,A_Capita_Per_GDP,A_Score,Y_Score,Y_Score_Per_Capita,A_Score_Per_Capita,country,latitude,longitude
1726,1726,2129,Ukraine,UKR,2020,Summer,44132049.0,1.655720e+11,3751.740723,1.0,...,3.330913e-06,1.147537e-10,8.878312e-10,260.0,27.0,1.634520e+06,1.697386e+05,UA,48.379433,31.165580
1611,1611,2107,Thailand,THA,2020,Summer,71475664.0,4.996818e+11,6990.935503,1.0,...,4.896772e-07,4.002548e-12,7.004458e-11,63.0,4.0,1.786892e+07,1.134534e+06,TH,15.870032,100.992541
1346,1346,2087,Poland,POL,2020,Summer,37899070.0,5.994492e+11,15816.989400,4.0,...,7.519973e-06,2.335477e-11,4.754365e-10,517.0,27.0,1.403669e+06,7.330574e+04,PL,51.919438,19.145136
1369,1369,2125,Qatar,QAT,2020,Summer,2760385.0,1.444114e+11,52315.660080,2.0,...,2.898146e-06,2.077399e-11,5.539730e-11,14.0,7.0,3.943407e+05,1.971704e+05,QA,25.354826,51.183884
1807,1807,2130,Uzbekistan,UZB,2020,Summer,34232050.0,5.989431e+10,1749.655815,3.0,...,1.080858e-06,8.348039e-11,6.177549e-10,65.0,11.0,3.112005e+06,5.266469e+05,UZ,41.377491,64.585262


In [44]:
# Figure 1 map with choropleth and scatter geo
import plotly.express as px
fig = px.choropleth(df, 
                    locations="Country_Code",
                    color="GDP",
                    hover_name="Country_Name",
                    animation_frame="Year",
                    hover_data=["Population","GDP", "GDP_Per_Capita","Y_Total","A_Total"],
                    range_color=[0,5000000000000],
                    color_continuous_scale=px.colors.sequential.Plasma_r)
fig2 = px.scatter_geo(df, lat="latitude", 
                    lon="longitude",
                    size="A_Total",           
                    hover_name= "A_Total",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    animation_frame='Year'
                    )

fig.add_trace(fig2.data[0])
for i, frame in enumerate(fig.frames):
    fig.frames[i].data += (fig2.frames[i].data[0],)
fig.show()
#pyo.plot(fig, filename='map.html', auto_open=True)

In [45]:
# Import necessary libraries
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import numpy as np

# Create figure
fig = go.Figure()

# Set default year
selected_year = 2020

# Define function to sort data by GDP and Total_Score
def sort_data_by_selected_year(selected_year):
    df_filtered_pop = df[df['Year'] == selected_year].sort_values(by='Population', ascending=True).tail(15)
    df_filtered_gdp = df[df['Year'] == selected_year].sort_values(by='GDP', ascending=True).tail(15)
    df_filtered_gdp_per_capita = df[(df['Year'] == selected_year) & (df['Population'] >= 1000000)].sort_values(by='GDP_Per_Capita', ascending=True).tail(15)
    return df_filtered_pop, df_filtered_gdp, df_filtered_gdp_per_capita

# Create two subplots
fig = sp.make_subplots(rows=1, cols=3)

df_filtered_pop, df_filtered_gdp, df_filtered_gdp_per_capita = sort_data_by_selected_year(df['Year'].max())

# Add first chart to subplot 1
df_filtered_pop = df[df['Year'] == selected_year].sort_values(by='Population', ascending=True).tail(10)
fig.add_trace(go.Bar(
    y=df_filtered_pop['Country_Name'],
    x=df_filtered_pop['Population'],
    name='Population',
    marker_color='red',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=1)
# Add title to x-axis of subplot 1
fig.update_xaxes(title_text='Top Population', row=1, col=1)



# Add second chart to subplot 2
df_filtered_gdp = df[df['Year'] == selected_year].sort_values(by='GDP', ascending=True).tail(10)
fig.add_trace(go.Bar(
  y=df_filtered_gdp['Country_Name'],
    x=df_filtered_gdp['GDP'],
    name='GDP',
    marker_color='blue',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=2)
# Add title to x-axis of subplot 2
fig.update_xaxes(title_text='Top GDP', row=1, col=2)



# Add third chart to subplot 3
filtered_gdp_per_capita = df[df['Year'] == selected_year].sort_values(by='GDP_Per_Capita', ascending=True).tail(10)
fig.add_trace(go.Bar(
    y=df_filtered_gdp_per_capita['Country_Name'],
    x=df_filtered_gdp_per_capita['GDP_Per_Capita'],
    name='GDP Per Capita',
    marker_color='green',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=3)
# Add title to x-axis of subplot 3
fig.update_xaxes(title_text='GDP Per Capita', row=1, col=3)



# Update layout
fig.update_layout(title=f'Country Population, GDP, and Medals in {df["Year"].max()}',
                  height=600, width=1000,
                  barmode='group',
                  showlegend=False)

# Create dropdown list of years
years = np.sort(df['Year'].unique())[::-1]
dropdown_buttons = []
for year in years:
    df_filtered_pop, df_filtered_gdp, df_filtered_gdp_per_capita = sort_data_by_selected_year(year)
    dropdown_buttons.append(dict(
        label=str(year),
        method="update",
        args=[{
                "x": [df_filtered_pop['Population'],df_filtered_gdp['GDP'], df_filtered_gdp_per_capita['GDP_Per_Capita']],
                "y": [df_filtered_pop['Country_Name'], df_filtered_gdp['Country_Name'], df_filtered_gdp_per_capita['Country_Name']],
                "type": "bar",
                "orientation": "h",
                "marker_color": ['red','blue','green'],
                "name": ['Population', 'GDP', 'GDP Per Capita']
              },
              {
                "title": f"Country Population, GDP, and Medal Score in {year}"
            }]
            ))

# Add dropdown
fig.update_layout(
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=0.1,
        xanchor="left",
        y=1.1,
        yanchor="top"
    )]
)

# Add annotation
fig.update_layout(
    annotations=[
        dict(text="Select Year:", showarrow=False,
        x=0, y=1.085, yref="paper", align="left")
    ]
)

fig.show()
#pyo.plot(fig, filename='barchart.html', auto_open=True)

In [46]:
fig = go.Figure()

import plotly.graph_objs as go

# Create the dropdown menu options
options = [{'label': 'Population', 'value': 'Population'},
           {'label': 'GDP', 'value': 'GDP'},
           {'label': 'GDP_Per_Capita', 'value': 'GDP_Per_Capita'}]

# Define the data for the initial bar chart (using GDP_Per_Capita as default)
data = [go.choropleth(df, 
                    locations="Country_Code",
                    color="GDP",
                    hover_name="Country_Name",
                    animation_frame="Year",
                    hover_data=["Population","GDP", "GDP_Per_Capita","Y_Total","A_Total"],
                    colorscale='Blues',
                    autocolorscale=False,
                    reversescale=True,
                    marker_line_color='darkgray']
# Define the layout
layout = go.Layout(title='Country Data')

# Define the figure
fig = go.Figure(data=data, layout=layout)

        # Define the dropdown menu
dropdown = go.layout.Updatemenu(
    buttons=list([
        dict(
            args=[{'color': [df['Population']]}],
            label='Population',
            method='update'
        ),
        dict(
            args=[{'color': [df['GDP']]}],
            label='GDP',
            method='update'
        ),
        dict(
            args=[{'color': [df['GDP_Per_Capita']]}],
            label='GDP_Per_Capita',
            method='update'
        )
        
    ]),
    direction='down',
    showactive=True,
    x=0.1,
    y=1.2
)

        
        
fig = px.choropleth(df, 
                    locations="Country_Code",
                    color="GDP",
                    hover_name="Country_Name",
                    animation_frame="Year",
                    hover_data=["Population","GDP", "GDP_Per_Capita","Y_Total","A_Total"],
                    range_color=[0,5000000000000],
                    color_continuous_scale=px.colors.sequential.Plasma_r)
fig2 = px.scatter_geo(df, lat="latitude", 
                    lon="longitude",
                    size="A_Total",           
                    hover_name= "A_Total",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    animation_frame='Year'
                    )

fig.add_trace(fig2.data[0])
for i, frame in enumerate(fig.frames):
    fig.frames[i].data += (fig2.frames[i].data[0],)
 


# Add the dropdown menu to the layout
fig.update_layout(showlegend=False, updatemenus=[dropdown])

# Display the figure
fig.show()
# pyo.plot(fig, filename='myplot.html', auto_open=True)

SyntaxError: invalid syntax (1756465398.py, line 20)

In [47]:
# Define the data for the initial choropleth map (using GDP_Per_Capita as default)
data = [go.Choropleth(
            locations=df['iso_alpha'],
            z=df['gdpPercap'],
            text=df['country'],
            colorscale='Blues',
            autocolorscale=False,
            reversescale=True,
            marker_line_color='darkgray',
            marker_line_width=0.5,
            colorbar_title='GDP per capita',
        )]

# Define the layout for the choropleth map
layout_choropleth = go.Layout(
    title='Country Data',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    coloraxis=dict(
        colorbar=dict(
            title='GDP per capita'
        )
    )
)

# Add the initial choropleth map to the figure
fig.add_trace(go.Figure(data=data, layout=layout_choropleth))

# Define the data for the scatter_geo plot (using area total as default)
data_scatter_geo = [go.Scattergeo(
                        lon=df['longitude'],
                        lat=df['latitude'],
                        text=df['country'],
                        marker=dict(
                            size=df['pop']/1e6,
                            sizemode='area',
                            sizeref=200,
                            color=df['area'],
                            colorscale='Viridis',
                            colorbar=dict(
                                title='Area'
                            ),
                            line=dict(width=0.5, color='rgb(40,40,40)'),
                            opacity=0.8,
                        ),
                        name='Area Total'
                    )]

# Define the layout for the scatter_geo plot
layout_scatter_geo = go.Layout(
    title='Country Data',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    showlegend=True,
    legend=dict(x=0, y=1)
)

# Add the initial scatter_geo plot to the figure
fig.add_trace(go.Figure(data=data_scatter_geo, layout=layout_scatter_geo))

# Define the callback function for the dropdown menu
def update_figure(selected_option):
    if selected_option == 'Population':
        fig.update_traces({'locations': df['iso_alpha'], 'z': df['pop'], 'text': df['country']}, selector=dict(type='choropleth'))
        fig.update_traces({'lat': df['latitude'], 'lon': df['longitude'], 'marker.size': df['pop']/1e6, 'marker.color': df['pop']}, selector=dict(type='scattergeo'))
        fig.update_layout(title='Population Data', coloraxis=dict(colorbar=dict(title='Population')))
    elif selected_option == 'GDP':
        fig.update_traces({'locations': df['iso_alpha'], 'z': df['gdpPercap'], 'text': df['country']}, selector=dict(type='choropleth'))
        fig.update_traces({'lat

SyntaxError: EOL while scanning string literal (2357401672.py, line 75)

In [48]:
import plotly.graph_objs as go
import plotly.subplots as sp
import plotly.express as px

pop = df.sort_values(by=['Year', 'GDP_Per_Capita'], ascending=True)
pop = pop.groupby('Year').tail(15)

fig = sp.make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'choropleth'}]])

fig.add_trace(go.Bar(x=pop['GDP_Per_Capita'], y=pop['Country_Name'], 
                     marker=dict(color=pop['GDP_Per_Capita'], 
                                 colorscale=px.colors.sequential.Plasma), 
                     orientation='h'), 
                     row=1, col=1),

fig.add_trace(go.Choropleth(locations=df_filtered_gdp['Country_Code'], 
                            z=df_filtered_gdp['GDP'], 
                            text=df_filtered_gdp['Country_Name'], 
                            colorscale='Blues_r', 
                            autocolorscale=False, 
                            reversescale=True, 
                            marker_line_color='darkgray', 
                            marker_line_width=0.5, 
                            colorbar_tickprefix='$', 
                            colorbar_title='GDP<br>(in billions USD)'), 
              row=1, col=2)

fig.update_layout(title_text='Top 10 Countries with Largest Population')

fig.show()

In [49]:
pop

,index,Unnamed: 0,Country_Name,Country_Code,Year,Season,Population,GDP,GDP_Per_Capita,Y_Gold,...,A_Capita_Per_Medal,Y_Capita_Per_GDP,A_Capita_Per_GDP,A_Score,Y_Score,Y_Score_Per_Capita,A_Score_Per_Capita,country,latitude,longitude
556,556,13,France,FRA,1960,Winter,46649927.0,6.222548e+10,1333.881573,1.0,...,1.714901e-07,4.821176e-11,1.285647e-10,12.0,5.0,9.329985e+06,3.887494e+06,FR,46.227638,2.213749
555,555,12,France,FRA,1960,Summer,46649927.0,6.222548e+10,1333.881573,0.0,...,1.071813e-07,8.035294e-11,8.035294e-11,7.0,7.0,6.664275e+06,6.664275e+06,FR,46.227638,2.213749
1727,1727,14,United Kingdom,GBR,1960,Summer,52400000.0,7.323397e+10,1397.594803,2.0,...,3.816794e-07,2.730973e-10,2.730973e-10,30.0,30.0,1.746667e+06,1.746667e+06,GB,55.378051,-3.435973
703,703,57,Iceland,ISL,1960,NaN,175574.0,2.484341e+08,1414.982269,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,IS,64.963051,-19.020835
1224,1224,25,Norway,NOR,1960,Winter,3581239.0,5.163272e+09,1441.755660,3.0,...,1.954631e-06,1.162054e-09,1.355730e-09,18.0,15.0,2.387493e+05,1.989577e+05,NO,60.472024,8.468946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,1784,2092,United States,USA,2020,Summer,331501080.0,2.106047e+13,63530.633480,39.0,...,5.508278e-06,5.365501e-12,8.670270e-11,3836.0,232.0,1.428884e+06,8.641843e+04,US,37.090240,-95.712891
1252,1252,2084,Norway,NOR,2020,Summer,5379475.0,3.621983e+11,67329.677790,4.0,...,6.561979e-05,2.208735e-11,9.746042e-10,732.0,18.0,2.988597e+05,7.349010e+03,NO,60.472024,8.468946
794,794,2095,Ireland,IRL,2020,Summer,4985382.0,4.258523e+11,85420.190860,2.0,...,5.215247e-06,9.392928e-12,6.105403e-11,48.0,8.0,6.231728e+05,1.038621e+05,IE,53.412910,-8.243890
1060,1060,2123,Luxembourg,LUX,2020,Summer,630419.0,7.399259e+10,117370.496900,0.0,...,3.172493e-06,0.000000e+00,2.702973e-11,4.0,0.0,0.000000e+00,1.576048e+05,LU,49.815273,6.129583


In [50]:
# Multiple Plots
fig = go.Figure()
selected_year = 2020


df_filtered_gdp = df[df['Year'] == selected_year].sort_values(by='GDP', ascending=True).tail(15)

fig = sp.make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'choropleth'}]])

pop = df.sort_values(by=['Year','Population'], ascending=True)
pop = pop.tail(20)
pop['Population'] = pop['Population']/1000000

df_filtered_gdp['GDP'] = df_filtered_gdp['GDP']/10000000000

fig.add_trace(go.Bar(x=df_filtered_gdp['GDP'],
                     y=df_filtered_gdp['Country_Name'], 
                     marker_color='indianred',
                     name='GDP',
                     orientation='h'))
fig.add_trace(go.Bar(x=df_filtered_gdp['A_Total'], 
                     y=df_filtered_gdp['Country_Name'], 
                     marker_color='lightsalmon',
                     name='Accumulated Medals',
                     orientation='h'),
                      row=1, col=1)

fig.add_trace(go.Choropleth(locations=df['Country_Code'],
                    z=df['GDP'],
                    text=df['Country_Name'],
                    colorscale='Viridis_r',
                    colorbar_title="GDP in USD")
                                   , row=1, col=2)

fig.show()
pyo.plot(fig, filename='myplot.html', auto_open=True)

'myplot.html'

In [51]:
import plotly.graph_objs as go

# Select the 10 most populous countries
pop = df.sort_values('Population', ascending=True)

# Scale accumulated medals by 100,000 for better visualization
pop['Population'] = pop['Population'] / 10000000

# Create a horizontal bar chart with Plotly
fig = go.Figure()
fig.add_trace(go.Bar(x=pop['Population'], 
                     y=pop['Country_Name'], 
                     marker_color='indianred',
                     name='Population by 1M',
                     orientation='h'))
fig.add_trace(go.Bar(x=pop['A_Total'], 
                     y=pop['Country_Name'], 
                     marker_color='lightsalmon',
                     name='Accumulated Medals',
                     orientation='h'))

# Customize the layout
fig.update_layout(title='Accumulated Medals vs Population (Top 10 Countries)',
                  xaxis_title='Population',
                  yaxis_title=None,
                  barmode='group',
                  height=600,
                  width=800)

# Show the figure
fig.show()

In [52]:
import matplotlib.pyplot as plt
def create_bar_chart(option):
    # Sort data by selected option in descending order
    sorted_df = df.sort_values(option, ascending=True)
    # Get top 10 values for selected option
    top_df = sorted_df
    # Create bar chart
    plt.barh(top_df['Country_Name'], top_df[option])
    plt.xlabel(option)
    plt.title(f'Top 10 Countries by {option}')
    plt.show()

# Create drop down menu to select option
from ipywidgets import interact, Dropdown

options = Dropdown(options=['GDP_Per_Capita', 'Population', 'GDP'], value='GDP_Per_Capita', description='Option:')

# Call create_bar_chart function when option is changed
interact(create_bar_chart, option=options);

interactive(children=(Dropdown(description='Option:', options=('GDP_Per_Capita', 'Population', 'GDP'), value='…

In [53]:
import plotly.graph_objs as go

# Create the dropdown menu options
options = [{'label': 'Population', 'value': 'Population'},
           {'label': 'GDP', 'value': 'GDP'},
           {'label': 'GDP_Per_Capita', 'value': 'GDP_Per_Capita'}]

# Define the data for the initial bar chart (using GDP_Per_Capita as default)
data = [go.Bar(x=df['Country_Name'], y=df['Population'])]

# Define the layout
layout = go.Layout(title='Country Data', xaxis_title='Country', yaxis_title='GDP_Per_Capita')

# Define the figure
fig = go.Figure(data=data, layout=layout)

# Define the dropdown menu
dropdown = go.layout.Updatemenu(
    buttons=list([
        dict(
            args=[{'y': [df['Population']]}],
            label='Population',
            method='update'
        ),
        dict(
            args=[{'y': [df['GDP']]}],
            label='GDP',
            method='update'
        ),
        dict(
            args=[{'y': [df['GDP_Per_Capita']]}],
            label='GDP_Per_Capita',
            method='update'
        )
        
    ]),
    direction='down',
    showactive=True,
    x=0.1,
    y=1.2
)

# Add the dropdown menu to the layout
fig.update_layout(showlegend=False, updatemenus=[dropdown])

# Display the figure
fig.show()

In [54]:
fig = go.Figure()

df_filtered_gdp = df.sort_values(by='GDP', ascending=True)
fig = sp.make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'choropleth'}]])

df_filtered_gdp['GDP'] = df_filtered_gdp['GDP']/10000000000

fig.add_trace(go.Bar(x=df_filtered_gdp['GDP'],
                     y=df_filtered_gdp['Country_Name'], 
                     marker_color='indianred',
                     name='GDP',
                     orientation='h'))
fig.add_trace(go.Bar(x=df_filtered_gdp['A_Total'], 
                     y=df_filtered_gdp['Country_Name'], 
                     marker_color='lightsalmon',
                     name='Accumulated Medals',
                     orientation='h'),
                      row=1, col=1)

fig.add_trace(go.Choropleth(locations=df['Country_Code'],
                    z=df['GDP'],
                    text=df['Country_Name'],
                    colorscale='Viridis_r')
                                   , row=1, col=2)

fig.show()
pyo.plot(fig, filename='myplot.html', auto_open=True)

'myplot.html'

In [55]:
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.offline as pyo

selected_year = 2020

df_filtered_gdp = df[df['Year']==selected_year].sort_values(by='GDP', ascending=True)
df_filtered_gdp = df_filtered_gdp.tail(10)

fig = sp.make_subplots(rows=1, cols=2, specs = [[{'type': 'bar'}, {'type': 'choropleth'}]])

df_filtered_gdp['GDP'] = df_filtered_gdp['GDP']/10000000000

colors = ['#FFD700', '#C0C0C0', '#CD7F32']  # gold, silver, bronze

fig.add_trace(go.Bar(x=df_filtered_gdp['A_Gold'], 
                     y=df_filtered_gdp['Country_Name'], 
                     name='Gold',
                     orientation='h',
                     marker=dict(color=colors[0])),
              row=1, col=1)

fig.add_trace(go.Bar(x=df_filtered_gdp['A_Silver'], 
                     y=df_filtered_gdp['Country_Name'], 
                     name='Silver',
                     orientation='h',
                     marker=dict(color=colors[1])),
              row=1, col=1)

fig.add_trace(go.Bar(x=df_filtered_gdp['A_Bronze'], 
                     y=df_filtered_gdp['Country_Name'], 
                     name='Bronze',
                     orientation='h',
                     marker=dict(color=colors[2])),
              row=1, col=1)

fig.add_trace(go.Bar(x=df_filtered_gdp['GDP'],
                     y=df_filtered_gdp['Country_Name'], 
                     marker_color='blue',
                     name='GDP',
                     orientation='h'),
              row=1, col=1)

fig.add_trace(go.Choropleth(locations=df['Country_Code'],
                            z=df['GDP'],
                            text=df['Country_Name'],
                            colorscale='Viridis_r'),
              row=1, col=2)

fig.update_layout(barmode='stack')

pyo.plot(fig, filename='myplot.html', auto_open=True)
fig.show()

In [56]:
# Multiple Plots
fig = go.Figure()
selected_year = 2020


df_filtered_gdp = df[df['Year'] == selected_year].sort_values(by='GDP', ascending=True).tail(15)

fig = sp.make_subplots(
    rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'choropleth'}]], 
    column_widths=[0.3, 0.7])

pop = df.sort_values(by=['Year','Population'], ascending=True)
pop = pop.tail(20)
pop['Population'] = pop['Population']/1000000

df_filtered_gdp['GDP'] = df_filtered_gdp['GDP']/10000000000
fig.add_trace(go.Bar(x=df_filtered_gdp['A_Total'], 
                     y=df_filtered_gdp['Country_Name'], 
                     marker_color='#479B55',
                     name='Accumulated Medals',
                     orientation='h')),
fig.add_trace(go.Bar(x=df_filtered_gdp['GDP'],
                     y=df_filtered_gdp['Country_Name'], 
                     marker_color='#636EFA',
                     name='GDP',
                     orientation='h'),
                     row=1, col=1)

fig.add_trace(go.Choropleth(locations=df['Country_Code'],
                    z=df['GDP'],
                    text=df['Country_Name'],
                    colorscale='Viridis_r',
                    colorbar_title="GDP in USD")
                                   , row=1, col=2)

fig.show()
pyo.plot(fig, filename='myplot.html', auto_open=True)

'myplot.html'

In [57]:
# Multiple Plots
fig = go.Figure()
selected_year = 2020

df_filtered_gdp = df[df['Year'] == selected_year].sort_values(by='GDP', ascending=False).head(30)

fig = sp.make_subplots(rows=2, cols=1, specs=[[{'type':'choropleth'}], [{'type':'bar'}]], row_heights=[0.7, 0.3])

pop = df.sort_values(by=['Year','Population'], ascending=False)
pop['Population'] = pop['Population']/1000000

df_filtered_gdp['GDP'] = df_filtered_gdp['GDP']/10000000000
df_filtered_gdp['GDP_Per_Capita'] = df_filtered_gdp['GDP_Per_Capita']/100

fig.add_trace(go.Bar(y=df_filtered_gdp['GDP'], 
                     x=df_filtered_gdp['Country_Name'], 
                     marker_color='#479B55',
                     name='GDP'))
fig.add_trace(go.Bar(y=df_filtered_gdp['GDP_Per_Capita'], 
                     x=df_filtered_gdp['Country_Name'], 
                     marker_color='#72B7B2',
                     name='GDP Per Capita'))

fig.add_trace(go.Bar(y=df_filtered_gdp['A_Total'],
                     x=df_filtered_gdp['Country_Name'], 
                     marker_color='#636EFA',
                     name='Accumulated Medals'),
                     row=2, col=1)

fig.add_trace(go.Choropleth(locations=df['Country_Code'],
                    z=df['GDP'],
                    text=df['Country_Name'],
                    colorscale='Viridis_r',
                    colorbar_title="GDP in USD"),
                                   row=1, col=1)

fig.show()
pyo.plot(fig, filename='myplot.html', auto_open=True)

'myplot.html'

In [58]:
import plotly.graph_objects as go

colors = ['#FFD700', '#C0C0C0', '#CD7F32']  # gold, silver, bronze

fig = go.Figure()

fig.add_trace(go.Bar(x=df_filtered_gdp['A_Gold'], 
                     y=df_filtered_gdp['Country_Name'], 
                     name='Gold',
                     orientation='h',
                     marker=dict(color=colors[0])))

fig.add_trace(go.Bar(x=df_filtered_gdp['A_Silver'], 
                     y=df_filtered_gdp['Country_Name'], 
                     name='Silver',
                     orientation='h',
                     marker=dict(color=colors[1])))

fig.add_trace(go.Bar(x=df_filtered_gdp['A_Bronze'], 
                     y=df_filtered_gdp['Country_Name'], 
                     name='Bronze',
                     orientation='h',
                     marker=dict(color=colors[2])))

fig.update_layout(barmode='stack')
fig.show()

In [59]:
fig = px.colors.qualitative.swatches()
print(px.colors.qualitative.T10)
fig.show()

['#4C78A8', '#F58518', '#E45756', '#72B7B2', '#54A24B', '#EECA3B', '#B279A2', '#FF9DA6', '#9D755D', '#BAB0AC']


In [60]:
colorscales = px.colors.named_colorscales()

In [61]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Animated GDP and population over decades'),
    html.P("Select an animation:"),
    dcc.RadioItems(
        id='selection',
        options=["GDP - Scatter", "Population - Bar"],
        value='GDP - Scatter',
    ),
    dcc.Loading(dcc.Graph(id="graph"), type="cube")
])


@app.callback(
    Output("graph", "figure"), 
    Input("selection", "value"))
def display_animated_graph(selection):
    df = px.data.gapminder() # replace with your own data source
    animations = {
        'GDP - Scatter': px.scatter(
            df, x="gdpPercap", y="lifeExp", animation_frame="year", 
            animation_group="country", size="pop", color="continent", 
            hover_name="country", log_x=True, size_max=55, 
            range_x=[100,100000], range_y=[25,90]),
        'Population - Bar': px.bar(
            df, x="continent", y="pop", color="continent", 
            animation_frame="year", animation_group="country", 
            range_y=[0,4000000000]),
    }
    return animations[selection]


app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


SystemExit: 1

C:\Users\ponen\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3561: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [ ]:
df

In [ ]:
# Import necessary libraries
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import numpy as np

# Create figure
fig = go.Figure()

# Set default year
selected_year = 2020

# Define function to sort data by GDP and Total_Score
def sort_data_by_selected_year(selected_year):
    df_filtered_pop = df[df['Year'] == selected_year].sort_values(by='Population', ascending=True).tail(15)
    df_filtered_gdp = df[df['Year'] == selected_year].sort_values(by='GDP', ascending=True).tail(15)
    df_filtered_gdp_per_capita = df[(df['Year'] == selected_year) & (df['Population'] >= 1000000)].sort_values(by='GDP_Per_Capita', ascending=True).tail(15)
    df_filtered_Y_Capita_Per_Medal = df[df['Year'] == selected_year].sort_values(by='Y_Capita_Per_Medal', ascending=True).tail(15)
    df_filtered_Y_Capita_Per_GDP = df[df['Year'] == selected_year].sort_values(by='Y_Capita_Per_GDP', ascending=True).tail(15)

    return df_filtered_pop, df_filtered_gdp, df_filtered_gdp_per_capita,df_filtered_Y_Capita_Per_Medal,df_filtered_Y_Capita_Per_GDP

# Create two subplots
fig = sp.make_subplots(rows=2, cols=3)

df_filtered_pop, df_filtered_gdp, df_filtered_gdp_per_capita,df_filtered_Y_Capita_Per_Medal,df_filtered_Y_Capita_Per_GDP = sort_data_by_selected_year(df['Year'].max())

# Add first chart to subplot 1
df_filtered_pop = df[df['Year'] == selected_year].sort_values(by='Population', ascending=True).tail(10)
fig.add_trace(go.Bar(
    y=df_filtered_pop['Country_Name'],
    x=df_filtered_pop['Population'],
    name='Population',
    marker_color='red',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=1)
# Add title to x-axis of subplot 1
fig.update_xaxes(title_text='Top Population', row=1, col=1)



# Add second chart to subplot 2
df_filtered_gdp = df[df['Year'] == selected_year].sort_values(by='GDP', ascending=True).tail(10)
fig.add_trace(go.Bar(
  y=df_filtered_gdp['Country_Name'],
    x=df_filtered_gdp['GDP'],
    name='GDP',
    marker_color='blue',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=2)
# Add title to x-axis of subplot 2
fig.update_xaxes(title_text='Top GDP', row=1, col=2)



# Add third chart to subplot 3
filtered_gdp_per_capita = df[df['Year'] == selected_year].sort_values(by='GDP_Per_Capita', ascending=True).tail(10)
fig.add_trace(go.Bar(
    y=df_filtered_gdp_per_capita['Country_Name'],
    x=df_filtered_gdp_per_capita['GDP_Per_Capita'],
    name='GDP Per Capita',
    marker_color='green',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=3)
# Add title to x-axis of subplot 3
fig.update_xaxes(title_text='GDP Per Capita', row=1, col=3)


# Add first chart to subplot 4
df_filtered_Y_Capita_Per_Medal = df[df['Year'] == selected_year].sort_values(by='Y_Capita_Per_Medal', ascending=True).tail(10)
fig.add_trace(go.Bar(
    y=df_filtered_Y_Capita_Per_Medal['Country_Name'],
    x=df_filtered_Y_Capita_Per_Medal['Y_Capita_Per_Medal'],
    name='Y_Capita_Per_Medal',
    marker_color='red',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=1)
# Add title to x-axis of subplot 4
fig.update_xaxes(title_text='Top Population', row=1, col=1)



# Add second chart to subplot 5
df_filtered_gdp_per_capita = df[df['Year'] == selected_year].sort_values(by='GDP', ascending=True).tail(10)
fig.add_trace(go.Bar(
  y=df_filtered_gdp['Country_Name'],
    x=df_filtered_gdp['GDP'],
    name='GDP',
    marker_color='blue',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=2)
# Add title to x-axis of subplot 5
fig.update_xaxes(title_text='Top GDP', row=1, col=2)


# Update layout
fig.update_layout(title=f'Country Population, GDP, and Medals in {df["Year"].max()}',
                  height=600, width=1000,
                  barmode='group',
                  showlegend=False)

# Create dropdown list of years
years = np.sort(df['Year'].unique())[::-1]
dropdown_buttons = []
for year in years:
    df_filtered_pop, df_filtered_gdp, df_filtered_gdp_per_capita,df_filtered_Y_Capita_Per_Medal,df_filtered_Y_Capita_Per_GDP = sort_data_by_selected_year(year)
    dropdown_buttons.append(dict(
        label=str(year),
        method="update",
        args=[{
                "x": [df_filtered_pop['Population'],df_filtered_gdp['GDP'], df_filtered_gdp_per_capita['GDP_Per_Capita'],df_filtered_Y_Capita_Per_Medal['Y_Capita_Per_Medal'],df_filtered_Y_Capita_Per_GDP['Y_Capita_Per_GDP']],
                "y": [df_filtered_pop['Country_Name'], df_filtered_gdp['Country_Name'], df_filtered_gdp_per_capita['Country_Name'],df_filtered_Y_Capita_Per_Medal['Country_Name'],df_filtered_Y_Capita_Per_GDP['Country_Name']],
                "type": "bar",
                "orientation": "h",
                "marker_color": ['red','blue','green', 'orange','yellow'],
                "name": ['Population', 'GDP', 'GDP Per Capita', 'Medal Per Capita', 'Medal Per GDP']
              },
              {
                "title": f"Country Population, GDP, and Medal Score in {year}"
            }]
            ))

# Add dropdown
fig.update_layout(
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=0.1,
        xanchor="left",
        y=1.1,
        yanchor="top"
    )]
)

# Add annotation
fig.update_layout(
    annotations=[
        dict(text="Select Year:", showarrow=False,
        x=0, y=1.085, yref="paper", align="left")
    ]
)

fig.show()
#pyo.plot(fig, filename='barchart.html', auto_open=True)

In [ ]:
# Import necessary libraries
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import numpy as np

# Create figure
fig = go.Figure()

# Set default year
selected_year = 2020

# Define function to sort data by GDP and Total_Score
def sort_data_by_selected_year(selected_year):
    df_filtered_pop = df[df['Year'] == selected_year].sort_values(by='Population', ascending=True).tail(15)
    df_filtered_gdp = df[df['Year'] == selected_year].sort_values(by='GDP', ascending=True).tail(15)
    df_filtered_gdp_per_capita = df[(df['Year'] == selected_year) & (df['Population'] >= 1000000)].sort_values(by='GDP_Per_Capita', ascending=True).tail(15)
    return df_filtered_pop, df_filtered_gdp, df_filtered_gdp_per_capita

# Create two subplots
fig = sp.make_subplots(rows=1, cols=3)

df_filtered_pop, df_filtered_gdp, df_filtered_gdp_per_capita = sort_data_by_selected_year(df['Year'].max())

# Add first chart to subplot 1
df_filtered_pop = df[df['Year'] == selected_year].sort_values(by='Population', ascending=True).tail(10)
fig.add_trace(go.Bar(
    y=df_filtered_pop['Country_Name'],
    x=df_filtered_pop['Population'],
    name='Population',
    marker_color='red',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=1)
# Add title to x-axis of subplot 1
fig.update_xaxes(title_text='Top Population', row=1, col=1)



# Add second chart to subplot 2
df_filtered_gdp = df[df['Year'] == selected_year].sort_values(by='GDP', ascending=True).tail(10)
fig.add_trace(go.Bar(
  y=df_filtered_gdp['Country_Name'],
    x=df_filtered_gdp['GDP'],
    name='GDP',
    marker_color='blue',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=2)
# Add title to x-axis of subplot 2
fig.update_xaxes(title_text='Top GDP', row=1, col=2)



# Add third chart to subplot 3
filtered_gdp_per_capita = df[df['Year'] == selected_year].sort_values(by='GDP_Per_Capita', ascending=True).tail(10)
fig.add_trace(go.Bar(
    y=df_filtered_gdp_per_capita['Country_Name'],
    x=df_filtered_gdp_per_capita['GDP_Per_Capita'],
    name='GDP Per Capita',
    marker_color='green',
    orientation='h',
    marker=dict(opacity=0.75)
), row=1, col=3)
# Add title to x-axis of subplot 3
fig.update_xaxes(title_text='GDP Per Capita', row=1, col=3)



# Update layout
fig.update_layout(title=f'Country Population, GDP, and Medals in {df["Year"].max()}',
                  height=600, width=1000,
                  barmode='group',
                  showlegend=False)

# Create dropdown list of years
years = np.sort(df['Year'].unique())[::-1]
dropdown_buttons = []
for year in years:
    df_filtered_pop, df_filtered_gdp, df_filtered_gdp_per_capita = sort_data_by_selected_year(year)
    dropdown_buttons.append(dict(
        label=str(year),
        method="update",
        args=[{
                "x": [df_filtered_pop['Population'],df_filtered_gdp['GDP'], df_filtered_gdp_per_capita['GDP_Per_Capita']],
                "y": [df_filtered_pop['Country_Name'], df_filtered_gdp['Country_Name'], df_filtered_gdp_per_capita['Country_Name']],
                "type": "bar",
                "orientation": "h",
                "marker_color": ['red','blue','green'],
                "name": ['Population', 'GDP', 'GDP Per Capita']
              },
              {
                "title": f"Country Population, GDP, and Medal Score in {year}"
            }]
            ))

# Add dropdown
fig.update_layout(
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=0.1,
        xanchor="left",
        y=1.1,
        yanchor="top"
    )]
)

# Add annotation
fig.update_layout(
    annotations=[
        dict(text="Select Year:", showarrow=False,
        x=0, y=1.085, yref="paper", align="left")
    ]
)

fig.show()
#pyo.plot(fig, filename='barchart.html', auto_open=True)

In [ ]:
import plotly.express as px

fig = go.Figure()
selected_year = 2020
def sort_data_by_selected_year(year):
    df_year = df[df['Year'] == year]
    df_year = df_year.sort_values(by='Total_Medals', ascending=False)
    return df_year
df_year = df[df['Year']==selected_year]

fig = px.scatter(df_year, y="GDP", x="Population", color="Country_Name", 
                 color_discrete_sequence=px.colors.qualitative.Dark24, hover_name="Country_Name")
fig.update_traces(marker_size=10, showlegend=True)
fig.update_layout(scattermode="group", scattergap=0.75)

# Update layout
fig.update_layout(title=f'Country Population, GDP, and Medals in {df_year["Year"].max()}',
                  height=600, width=1000,
                  barmode='group',
                  showlegend=False)

# Create dropdown list of years
years = np.sort(df_year['Year'].unique())[::-1]
dropdown_buttons = []
for year in years:
    dropdown_buttons.append(dict(
        label=str(year),
        method="update",
        args=[
              {"x": [df_year['Population']],
               "y": [df_year['GDP']],
               "title": f"Country Population, GDP, and Medal Score in {year}"
            }]
            ))

# Add dropdown
fig.update_layout(
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=0.1,
        xanchor="left",
        y=1.1,
        yanchor="top"
    )]
)

# Add annotation
fig.update_layout(
    annotations=[
        dict(text="Select Year:", showarrow=False,
        x=0, y=1.085, yref="paper", align="left")
    ]
)

fig.show()

In [62]:
# Import necessary libraries
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import numpy as np

# Create figure
fig = go.Figure()

# Set default year
selected_year = 2020
df_year = df[df['Year']==selected_year]
# Create two subplots
fig = sp.make_subplots(rows=1, cols=3)

# Add first chart to subplot 1
fig = px.scatter(df_year, y="GDP", x="Population", color="Country_Name", hover_name="Country_Name")
fig.update_traces(marker_size=10, showlegend=True)
fig.update_layout(scattermode="group", scattergap=0.5)



fig.show()

In [77]:
# Import necessary libraries
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import numpy as np

# Create figure
scatter_plot = go.Figure()

# Set default year
selected_year = 2016
df_year = df[df['Year']==selected_year]
# Create two subplots

# Add first chart to subplot 1
scatter_plot = px.scatter(df_year, y="GDP_Per_Capita", x="Y_Capita_Per_Medal", color="Country_Name", hover_name="Country_Name")
scatter_plot.update_traces(marker_size=10, showlegend=True)
scatter_plot.update_layout(scattermode="group", scattergap=0.5)


# Update layout
scatter_plot.update_layout(
    title={
        'text': "Performance by Population Size",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Medals Per Population",
    yaxis_title="GDP per Capita",
    legend_title="Country",
    font=dict(family="Arial", size=12),
    plot_bgcolor="#F9F9F9",
    hovermode="closest",
    margin=dict(l=50, r=50, t=50, b=50),
    paper_bgcolor="#F9F9F9",
    height=500,
    width=800,
)

    
scatter_plot.show()